In [69]:
import os
import torch
import pandas as pd
import requests
from dotenv import load_dotenv
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain



from langchain import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceEndpoint



## Data Import and preprocsiing

In [60]:
df = pd.read_csv('flipkart_review_data_2022_02.csv')
df = df[df['reviewDescription'].str.split(" ").str.len()>80]


## Model

In [3]:
def config():
    load_dotenv()

config()

In [68]:
repo_id = "google/gemma-7b-it"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    top_k=1,
    top_p=0.9,    
    temperature=0.1,
    max_new_tokens = 500,
    huggingfacehub_api_token=os.getenv('API_KEY')
)


In [5]:
# To check input given to model.
def gemma_template(prompt):
    text =  "<start_of_turn>user\n{}<end_of_turn>".format(prompt) + "\n<start_of_turn>model"    
    return text


In [74]:
print(llm(gemma_template("hi")))



Hi, 👋

It's nice to hear from you. What would you like to talk about today?


## Conversational Chain

#### Dose not work properly

In [42]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True    
)
print(conversation.predict(input=gemma_template(("HI"))))
conversation.predict(input="What is capital of india")
conversation.predict(input="What is my name")   



## Chains

### Single chain

In [40]:
# text = "What is the capital of India"
first_prompt = ChatPromptTemplate.from_template(gemma_template("What is the best name to describe a company that makes {product}?,just give me one name"))

In [70]:
chain_one = LLMChain(llm=llm,prompt=first_prompt)
chain_one

In [71]:
print(chain_one.run("Queen Size Sheet Set"))

### double chain

In [23]:
# text = "What is the capital of India"
first_prompt = ChatPromptTemplate.from_template(gemma_template("What is the best name to describe a company that makes {product}?,just give me one name"))
second_prompt = ChatPromptTemplate.from_template(gemma_template("Write a 20 words description for the following  company:{company_name}"))


In [24]:
chain_one = LLMChain(llm=llm, prompt=first_prompt)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [25]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [26]:
print(overall_simple_chain.run("Queen Size Sheet Set"))



> Entering new SimpleSequentialChain chain...


HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/google/gemma-7b-it (Request ID: kf2Q-0tPPpvzqTc9FyvON)

## Sequential Chains

In [8]:
first_prompt = ChatPromptTemplate.from_template(gemma_template('''Translate the following review to english: \n\n{Review}'''))

chain_one = LLMChain(llm=llm,prompt=first_prompt,output_key="English_Review")

In [10]:
second_prompt = ChatPromptTemplate.from_template(gemma_template("Can you summarize the following review in 1 sentence:"
                                                                "\n\n{English_Review}"))

chain_two = LLMChain(llm=llm,prompt=second_prompt,output_key="summary")

In [11]:
third_prompt = ChatPromptTemplate.from_template(gemma_template("What language is the following review:\n\n{Review}"))

chain_three = LLMChain(llm=llm, prompt=third_prompt,output_key="language")


In [12]:
fourth_prompt = ChatPromptTemplate.from_template(
                            gemma_template
                            (
                            "Write a follow up response to the following "
                            "summary in the specified language:"
                            "\n\nSummary: {summary}\n\nLanguage: {language}"
                            )
                        )


chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [22]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [72]:
review = df['reviewDescription'].iloc[0]
review

In [73]:
print(overall_chain.invoke(review))